## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json

import itertools
import math

from tqdm import tqdm

import pyDOE
import sys

from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render
import plotly.io as pio

import pickle
from matplotlib.pyplot import cm
import scienceplots

from utils import score_vals, param_to_list, list_to_param

# Mute warnings:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ax.plot.trace import optimization_trace_single_method

In [3]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [4]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10

## Start up client

In [5]:
Tag_client = AxClient()

[INFO 04-10 13:51:24] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [6]:
# Define bounds for each parameter type
int_bounds = {
    "z_sh": [5, 15], #NOTE - z_sh is actually half z_s (which must be even)
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-1, 1],
    "x_r2": [-1., 1.],
    # Cl is fixed as : 0.10-0.40
}

paper_vals = [6, 81, .476, 1.0]

# Number of copies for each parameter
num_copies = 3

# Generate parameters dynamically
parameters = []

# Add integer parameters
for name, bounds in int_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (num_copies-1-i), upper - i],  # adjust bounds to deal with order
            "value_type": "int",
        })

# Add float parameters
for name, bounds in float_bounds.items():
    lower, upper = bounds
    for i in range(num_copies):
        param_name = f"{name}" if i == 0 else f"{name}_{i}"
        parameters.append({
            "name": param_name,
            "type": "range",
            "bounds": [lower + (.01*(num_copies-1-i)), upper - .01*i],  # adjust bounds to deal with order
            "value_type": "float",
        })

# Add int / float parameters that make sure 

# Generate parameter constraints
parameter_constraints = []

# Ensure integer `z` values satisfy z_s - z_s_1 >= 1
for name in int_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 1")

# Ensure float `x` values satisfy x_s - x_s_1 >= 0.1
for name in float_bounds.keys():
    for i in range(num_copies-1):
        j = i+1
        param_i = f"{name}" if i == 0 else f"{name}_{i}"
        param_j = f"{name}_{j}"
        parameter_constraints.append(f"{param_i} - {param_j} >= 0.1")

# Create the experiment
Tag_client.create_experiment(
    name="gear_experiment",
    parameters=parameters,
    objectives={"Viable_Cross_Terms": ObjectiveProperties(minimize=False)},
    parameter_constraints=parameter_constraints,  # Add constraints here
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

# Print constraints for verification
# print("Parameter Constraints:")
# for constraint in parameter_constraints:
#     print(constraint)

[INFO 04-10 13:51:24] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[7, 15]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[6, 14]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[5, 13]), RangeParameter(name='z_r2', parameter_type=INT, range=[72, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[71, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 84]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[-0.98, 1.0]), RangeParameter(name='x_s_1', parameter_type=FLOAT, range=[-0.99, 0.99]), RangeParameter(name='x_s_2', parameter_type=FLOAT, range=[-1.0, 0.98]), RangeParameter(name='x_r2', parameter_type=FLOAT, range=[-0.98, 1.0]), RangeParameter(name='x_r2_1', parameter_type=FLOAT, range=[-0.99, 0.99]), RangeParameter(name='x_r2_2', parameter_type=FLOAT, range=[-1.0, 0.98])], parameter_constraints=[ParameterConstraint(-1.0*z_sh + 1.0*z_sh_1 <= -1

In [7]:
max_cross_terms = 4**(len(int_bounds) + len(float_bounds)+1) # plus Cl.
print(f'max_cross_terms: {max_cross_terms}')

max_cross_terms: 1024


In [8]:
# -1.0*z_p2 + 1.0*z_p2_1 <= -1.0
# -1.0*x_p2_1 + 1.0*x_p2 <= -0.01

In [9]:
parameters

[{'name': 'z_sh', 'type': 'range', 'bounds': [7, 15], 'value_type': 'int'},
 {'name': 'z_sh_1', 'type': 'range', 'bounds': [6, 14], 'value_type': 'int'},
 {'name': 'z_sh_2', 'type': 'range', 'bounds': [5, 13], 'value_type': 'int'},
 {'name': 'z_r2', 'type': 'range', 'bounds': [72, 86], 'value_type': 'int'},
 {'name': 'z_r2_1', 'type': 'range', 'bounds': [71, 85], 'value_type': 'int'},
 {'name': 'z_r2_2', 'type': 'range', 'bounds': [70, 84], 'value_type': 'int'},
 {'name': 'x_s',
  'type': 'range',
  'bounds': [-0.98, 1.0],
  'value_type': 'float'},
 {'name': 'x_s_1',
  'type': 'range',
  'bounds': [-0.99, 0.99],
  'value_type': 'float'},
 {'name': 'x_s_2',
  'type': 'range',
  'bounds': [-1.0, 0.98],
  'value_type': 'float'},
 {'name': 'x_r2',
  'type': 'range',
  'bounds': [-0.98, 1.0],
  'value_type': 'float'},
 {'name': 'x_r2_1',
  'type': 'range',
  'bounds': [-0.99, 0.99],
  'value_type': 'float'},
 {'name': 'x_r2_2',
  'type': 'range',
  'bounds': [-1.0, 0.98],
  'value_type': 'f

In [10]:
parameter_constraints

['z_sh - z_sh_1 >= 1',
 'z_sh_1 - z_sh_2 >= 1',
 'z_r2 - z_r2_1 >= 1',
 'z_r2_1 - z_r2_2 >= 1',
 'x_s - x_s_1 >= 0.1',
 'x_s_1 - x_s_2 >= 0.1',
 'x_r2 - x_r2_1 >= 0.1',
 'x_r2_1 - x_r2_2 >= 0.1']

### Run tests to solve for best possible score    

In [11]:
success = 0
while not success:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        success = 1
    except:
        print("Error: unable to find next trial.")

[WARNING 04-10 13:51:24] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 14.89518571617473, 'z_sh_1': 10.369652356927432, 'z_sh_2': 6.112244990158043, 'z_r2': 85.45675460448275, 'z_r2_1': 73.30946883394843, 'z_r2_2': 69.79065102927366, 'x_s': 0.09748274566605697, 'x_s_1': -0.18886407231912017, 'x_s_2': -0.8719280664250255, 'x_r2': 0.199709575176239, 'x_r2_1': -0.39235822202637793, 'x_r2_2': -0.9198320652917027}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[7, 15]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[6, 14]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[5, 13]), RangeParameter(name='z_r2', parameter_type=INT, range=[72, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[71, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 84]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x_s_1', parameter_type=FLOAT

In [12]:
parameterization

{'z_sh': 15,
 'z_sh_1': 10,
 'z_sh_2': 6,
 'z_r2': 85,
 'z_r2_1': 73,
 'z_r2_2': 70,
 'x_s': 0.09748274566605697,
 'x_s_1': -0.18886407231912017,
 'x_s_2': -0.8719280664250255,
 'x_r2': 0.199709575176239,
 'x_r2_1': -0.39235822202637793,
 'x_r2_2': -0.9198320652917027}

In [13]:
print(len(parameterization))
for name, value in parameterization.items():
    print(f"{name}: {value}")

12
z_sh: 15
z_sh_1: 10
z_sh_2: 6
z_r2: 85
z_r2_1: 73
z_r2_2: 70
x_s: 0.09748274566605697
x_s_1: -0.18886407231912017
x_s_2: -0.8719280664250255
x_r2: 0.199709575176239
x_r2_1: -0.39235822202637793
x_r2_2: -0.9198320652917027


In [14]:
Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True)))

[INFO 04-10 13:51:25] ax.service.ax_client: Completed trial 0 with data: {'Viable_Cross_Terms': (497, None)}.


In [15]:
list = param_to_list(parameterization, add_cl = True)
print(list)

[array([15, 10,  6]), array([85, 73, 70]), array([ 0.09748275, -0.18886407, -0.87192807]), array([ 0.19970958, -0.39235822, -0.91983207]), array([0.0002, 0.0003, 0.0004])]


NOTE - run the following through 'Exhausted' errors.

In [16]:
init_count = 0
init_trials = 24 # hand set based on output at initialization
while init_count < init_trials:
    try:
        parameterization, trial_index = Tag_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True)))
    except:
        print('Not this time')
    init_count = trial_index

[WARNING 04-10 13:51:25] ax.modelbridge.transforms.int_to_float: Unable to round {'z_sh': 10.277975211685598, 'z_sh_1': 7.425499702813699, 'z_sh_2': 4.6150120962522365, 'z_r2': 73.53261317911213, 'z_r2_1': 72.2971910484789, 'z_r2_2': 69.9037362192017, 'x_s': 0.46775089237838974, 'x_s_1': 0.2873858828656375, 'x_s_2': -0.5757044620998204, 'x_r2': 0.8096036698482931, 'x_r2_1': 0.3627705234289169, 'x_r2_2': 0.11083223871886738}to meet parameter constraints of SearchSpace(parameters=[RangeParameter(name='z_sh', parameter_type=INT, range=[7, 15]), RangeParameter(name='z_sh_1', parameter_type=INT, range=[6, 14]), RangeParameter(name='z_sh_2', parameter_type=INT, range=[5, 13]), RangeParameter(name='z_r2', parameter_type=INT, range=[72, 86]), RangeParameter(name='z_r2_1', parameter_type=INT, range=[71, 85]), RangeParameter(name='z_r2_2', parameter_type=INT, range=[70, 84]), RangeParameter(name='x_s', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x_s_1', parameter_type=FLOAT, ra

In [17]:
# a decent trial to check
params = ((5, 9, 10),
 (74, 75, 76),
 (-0.25, 0.0, 0.5),
 (-0.5, -0.25, 0.0))

vals = [np.array(x) for x in params]

comp_params_1 = list_to_param(vals)

In [18]:
comp_params_1

{'z_sh': 5,
 'z_sh_1': 9,
 'z_sh_2': 10,
 'z_r2': 74,
 'z_r2_1': 75,
 'z_r2_2': 76,
 'xs': -0.25,
 'xs_1': 0.0,
 'xs_2': 0.5,
 'xr2': -0.5,
 'xr2_1': -0.25,
 'xr2_2': 0.0}

In [19]:
score_vals(param_to_list(comp_params_1, add_cl = True))

677

In [20]:
Tag_client.generation_strategy.trials_as_df

,trial_index,arm_name,trial_status,generation_method,generation_node,Viable_Cross_Terms,z_sh,z_sh_1,z_sh_2,z_r2,z_r2_1,z_r2_2,x_s,x_s_1,x_s_2,x_r2,x_r2_1,x_r2_2
0,0,0_0,COMPLETED,Sobol,GenerationStep_0,497.0,15,10,6,85,73,70,0.097483,-0.188864,-0.871928,0.199710,-0.392358,-0.919832
1,1,1_0,COMPLETED,Sobol,GenerationStep_0,674.0,10,7,5,74,72,70,0.467751,0.287386,-0.575704,0.809604,0.362771,0.110832
2,2,2_0,COMPLETED,Sobol,GenerationStep_0,774.0,15,13,8,74,72,71,0.829612,-0.247872,-0.495559,-0.224213,-0.416648,-0.647247
3,3,3_0,COMPLETED,Sobol,GenerationStep_0,549.0,13,12,10,80,72,70,0.844803,0.096848,-0.642085,0.386873,0.090325,-0.983657
4,4,4_0,COMPLETED,Sobol,GenerationStep_0,438.0,15,13,11,79,76,74,0.796761,-0.435139,-0.606262,0.853792,0.437270,-0.567486
5,5,5_0,COMPLETED,Sobol,GenerationStep_0,40.0,15,10,6,85,84,80,-0.262288,-0.417469,-0.625430,0.895078,0.752916,-0.529921
6,6,6_0,COMPLETED,Sobol,GenerationStep_0,168.0,11,7,5,86,84,75,0.327855,-0.056006,-0.532340,0.519265,0.241159,0.050255
7,7,7_0,COMPLETED,Sobol,GenerationStep_0,257.0,11,9,5,84,83,71,0.966534,0.589077,-0.457383,0.640521,0.484115,0.093254
8,8,8_0,COMPLETED,Sobol,GenerationStep_0,463.0,9,8,5,81,75,71,0.245029,-0.333780,-0.988002,0.383967,-0.108446,-0.285527
9,9,9_0,COMPLETED,Sobol,GenerationStep_0,388.0,13,10,7,82,77,73,0.921953,0.195168,-0.002247,0.656009,0.270142,-0.063256


In [21]:
render(Tag_client.get_optimization_trace())

In [ ]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization, add_cl = True)))

[INFO 04-10 13:51:52] ax.service.ax_client: Generated new trial 25 with parameters {'z_sh': 15, 'z_sh_1': 14, 'z_sh_2': 5, 'z_r2': 72, 'z_r2_1': 71, 'z_r2_2': 70, 'x_s': 0.793432, 'x_s_1': -0.406846, 'x_s_2': -0.506846, 'x_r2': 0.09562, 'x_r2_1': -0.00438, 'x_r2_2': -0.10438} using model BoTorch.
[INFO 04-10 13:51:52] ax.service.ax_client: Completed trial 25 with data: {'Viable_Cross_Terms': (795, None)}.
[INFO 04-10 13:52:01] ax.service.ax_client: Generated new trial 26 with parameters {'z_sh': 13, 'z_sh_1': 12, 'z_sh_2': 7, 'z_r2': 76, 'z_r2_1': 75, 'z_r2_2': 70, 'x_s': 1.0, 'x_s_1': 0.9, 'x_s_2': 0.8, 'x_r2': -0.27042, 'x_r2_1': -0.37042, 'x_r2_2': -0.47042} using model BoTorch.
[INFO 04-10 13:52:01] ax.service.ax_client: Completed trial 26 with data: {'Viable_Cross_Terms': (712, None)}.
[INFO 04-10 13:52:11] ax.service.ax_client: Generated new trial 27 with parameters {'z_sh': 13, 'z_sh_1': 9, 'z_sh_2': 6, 'z_r2': 72, 'z_r2_1': 71, 'z_r2_2': 70, 'x_s': 1.0, 'x_s_1': 0.9, 'x_s_2': 0

In [ ]:
best_parameters, values = Tag_client.get_best_parameters()
for name, value in best_parameters.items():
    print(f"{name}: {value}")

[INFO 04-10 13:40:15] ax.modelbridge.base: Leaving out out-of-design observations for arms: 87_0


z_sh: 15
z_sh_1: 10
z_sh_2: 9
z_r2: 72
z_r2_1: 71
z_r2_2: 70
x_s: 0.9999999999960997
x_s_1: 0.8999999999149966
x_s_2: 0.7999999998281693
x_r2: -0.25155638920287665
x_r2_1: -0.351556389257909
x_r2_2: -0.9999999999177834


In [30]:
print(score_vals(param_to_list(best_parameters, add_cl = True)))

790


In [31]:
param_to_list(best_parameters)

[array([15, 10]),
 array([ 9, 72]),
 array([71, 70]),
 array([1. , 0.9]),
 array([ 0.8       , -0.25155639])]

In [32]:
render(Tag_client.get_optimization_trace())

In [27]:
for i in range(100):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-05 14:15:58] ax.service.ax_client: Generated new trial 147 with parameters {'z_sh': 8, 'z_sh_1': 10, 'z_sh_2': 5, 'z_p2': 33, 'z_p2_1': 31, 'z_p2_2': 31, 'z_r2': 80, 'z_r2_1': 85, 'z_r2_2': 82, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.629718, 'x_p1_1': 0.529718, 'x_p1_2': 0.429718, 'x_p2': 0.410605, 'x_p2_1': 0.310605, 'x_p2_2': 0.210605, 'x_r2': 0.981954, 'x_r2_1': 0.881954, 'x_r2_2': 0.781954} using model BoTorch.
[INFO 04-05 14:15:58] ax.service.ax_client: Completed trial 147 with data: {'Viable_Cross_Terms': (85, None)}.
[INFO 04-05 14:16:17] ax.service.ax_client: Generated new trial 148 with parameters {'z_sh': 12, 'z_sh_1': 12, 'z_sh_2': 6, 'z_p2': 34, 'z_p2_1': 30, 'z_p2_2': 25, 'z_r2': 86, 'z_r2_1': 76, 'z_r2_2': 83, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.869118, 'x_p1_1': 0.769118, 'x_p1_2': 0.669118, 'x_p2': 0.390927, 'x_p2_1': 0.290927, 'x_p2_2': 0.190927, 'x_r2': 0.794506, 'x_r2_1': 0.694506, 'x_r2_2': 0.594506} using model BoTorch.
[INFO 04-05 

In [44]:
render(Tag_client.get_optimization_trace())

In [ ]:
#Viable_Cross_Terms               133.0
z_sh = [ 7, 8, 9]
z_p2 = [28,31,26]
z_r2 = [82,74,83]
x_s = [ 0.5, 0.4, 0.3]
x_p1 = [ 1.0, 0.9, 0.8]
x_p2 = [-0.3,-0.4,-0.5]
x_r2 = [ 0.461185, 0.361185, 0.261185]

In [42]:
for i in range(800):
    parameterization, trial_index = Tag_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    Tag_client.complete_trial(trial_index=trial_index, raw_data=score_vals(param_to_list(parameterization)))

[INFO 04-06 08:44:12] ax.modelbridge.base: Leaving out out-of-design observations for arms: 729_0, 977_0, 489_0, 686_0, 775_0, 899_0
[INFO 04-06 08:46:47] ax.service.ax_client: Generated new trial 1029 with parameters {'z_sh': 15, 'z_sh_1': 11, 'z_sh_2': 5, 'z_p2': 30, 'z_p2_1': 34, 'z_p2_2': 25, 'z_r2': 79, 'z_r2_1': 81, 'z_r2_2': 84, 'x_s': 0.5, 'x_s_1': 0.4, 'x_s_2': 0.3, 'x_p1': 0.643009, 'x_p1_1': 0.543009, 'x_p1_2': 0.443009, 'x_p2': 0.5, 'x_p2_1': 0.4, 'x_p2_2': 0.3, 'x_r2': 1.5, 'x_r2_1': -0.9846, 'x_r2_2': -1.5} using model BoTorch.
[INFO 04-06 08:46:47] ax.service.ax_client: Completed trial 1029 with data: {'Viable_Cross_Terms': (0, None)}.
[INFO 04-06 08:46:47] ax.modelbridge.base: Leaving out out-of-design observations for arms: 729_0, 977_0, 489_0, 686_0, 775_0, 899_0
[INFO 04-06 08:51:00] ax.service.ax_client: Generated new trial 1030 with parameters {'z_sh': 7, 'z_sh_1': 6, 'z_sh_2': 5, 'z_p2': 35, 'z_p2_1': 34, 'z_p2_2': 25, 'z_r2': 80, 'z_r2_1': 85, 'z_r2_2': 84, 'x_s'

KeyboardInterrupt: 